# Machine Learning LAB 1 
Course 2022/23: F. Barbato, M. Mel, P. Zanuttigh

The notebook contains some simple tasks to be performed about **classification and regression**. <br>
Complete all the **required code sections** and **answer to all the questions**. <br>

### IMPORTANT for the evaluation score:
1. **Read carefully all cells** and **follow the instructions**
1. **Rerun all the code from the beginning** to obtain the results for the final version of your notebook, since this is the way we will do it before evaluating your notebooks.
2. Make sure to fill the code in the appropriate places **without modifying the template**, otherwise you risk breaking later cells.
3. Please **submit the jupyter notebook file (.ipynb)**, do not submit python scripts (.py) or plain text files. **Make sure that it runs fine with the restat&run all command** - otherwise points will be deduced.
4. **Answer the questions in the appropriate cells**, not in the ones where the question is presented.

## A) Classification of Day/Night

Place your **name** and **ID number** (matricola) in the cell below. <br>
Also recall to **save the file as Surname_Name_LAB1.ipynb** otherwise your homework could get lost
<br>

**Student name**: Mohammad Takafouyan<br>
**ID Number**: 

### Dataset description

The data was recorded using the new **Luxottica I-SEE glasses** in exterior conditions. These devices provide multiple **sensors mounted inside the glasses**, which can be accessed through a bluetooth connection. <br>
For the **classification** part of the notebook we will focus on the **UVA**, **UVB** and **pressure** sensors, with the goal of discriminating between **day and night** time.

![I-SEE Glasses](data/isee.png "I-SEE")

We first **import** all **the packages** that are needed.

In [1]:
import csv
from matplotlib import pyplot as plt
import numpy as np
from sklearn import linear_model, preprocessing

Change some global settings for layout purposes.

In [2]:
# if you are in the jupyter notebook environment you can change the 'inline' option with 'notebook' to get interactive plots
%matplotlib notebook
# change the limit on the line length and crop to 0 very small numbers, for clearer printing
np.set_printoptions(linewidth=500, suppress=True)

## A.1) Perceptron
In the following cells we will **implement** the **perceptron** algorithm and use it to learn a halfspace.

**TO DO (A.1.0):** **Set** the random **seed** using your **ID**. If you need to change it for testing add a constant explicitly, eg.: 1234567 + 1

In [3]:
IDnumber = 
np.random.seed(IDnumber)

Before proceding to the training steps, we **load the dataset and split it** in training and test set (the **training** set is **typically larger**, here we use a 75% training 25% test split).
The **split** is **performed after applying a random permutation** to the dataset, such permutation will **depend on the seed** you set above. Try different seeds to evaluate the impact of randomization.<br><br>
**DO NOT CHANGE THE PRE-WRITTEN CODE UNLESS OTHERWISE SPECIFIED**

In [4]:
def load_dset(filename, features=[2,3,9], label_id=-1, mode='clas'):
    # Load the dataset
    with open(filename, newline='\n') as f:
        raw_data = csv.reader(f, delimiter=',')
        header = next(raw_data)      
        print(f"Header: {header}\n")
        
        dataset = np.array(list(raw_data))
        print(f"Data shape: {dataset.shape}\n")
        print("Dataset Example:")
        print(dataset[0,...])                  

    X = dataset[:,features].astype(float)      # extract the selected features, default refers to the first part of the lab: [uva, uvb, pressure]
    if mode == 'clas':
        Y = dataset[:,label_id].astype(int)    # if we are in classification mode, get the labels from the provided index as indices
        Y = 2*Y-1                              # for the perceptron night --> -1, day --> 1
    else:
        Y = dataset[:,label_id].astype(float)  

    m = dataset.shape[0]
    print("\nNumber of samples loaded:", m)
    permutation = np.random.permutation(m)     

    X = X[permutation]
    Y = Y[permutation]
    
    return X, Y

In [5]:
# Load the dataset
X, Y = load_dset('lux.csv')

Header: ['rh', 'temp', 'uva', 'uvb', 'x', 'y', 'blueg', 'blueb', 'worn', 'pressure', 'timestamp', 'hour', 'minute', 'second', 'isnight']

Data shape: (2177, 15)

Dataset Example:
['23.846666666666668' '37.21333333333334' '5.577416666666666' '1.6235333333333333' '615.7777777777778' '686.0' '134.0' '117.0' '0.0' '1.000291690654385' '1650458376' '14' '39' '36' '0']

Number of samples loaded: 2177


We are going to differentiate (classify) between **class "1" (day)** and **class "-1" (night)**

# Split data in training and test sets



Given $m$ total data, denote with $m_{t}$ the part used for training. Keep $m_t$ data as training data, and $m_{test}:= m-m_{t}$. <br>
For instance one can take $m_t=0.75m$ of the data as training and $m_{test}=0.25m$ as testing. <br>
Let us define as define

$\bullet$ $S_{t}$ the training data set

$\bullet$ $S_{test}$ the testing data set


The reason for this splitting is as follows:

TRAINING DATA: The training data are used to compute the empirical loss
$$
L_S(h) = \frac{1}{m_t} \sum_{z_i \in S_{t}} \ell(h,z_i)
$$
which is used to estimate $h$ in a given model class ${\cal H}$.
i.e. 
$$
\hat{h} = {\rm arg\; min}_{h \in {\cal H}} \, L_S(h)
$$

TESTING DATA: The test data set can be used to estimate the performance of the final estimated model
$\hat h_{\hat d_j}$ using:
$$
L_{{\cal D}}(\hat h_{\hat d_j}) \simeq \frac{1}{m_{test}} \sum_{ z_i \in S_{test}} \ell(\hat h_{\hat d_j},z_i)
$$

**TO DO (A.1.1):** **Divide** the **data into training and test set** (**75%** of the data in the **first** set, **25%** in the **second** one). <br>
<br>
Notice that as is common practice in Statistics and Machine Learning, **we scale the data** (= each variable) so that it is centered **(zero mean)** and has **standard deviation equal to 1**. <br>
This helps in terms of numerical conditioning of the (inverse) problems of estimating the model (the coefficients of the linear regression in this case), as well as to give the same scale to all the coefficients.

In [6]:
# compute the splits
m = X.shape[0]
m_training = np.floor(0.75*m).astype(int)
print(m)
print(m_training)
# m_test is the number of samples in the test set (total-training)
m_test =  m - m_training

# X_training = instances for training set
X_training =  X[:m_training,:]
# Y_training = labels for the training set
Y_training =  Y[:m_training]

# X_test = instances for test set
X_test =   X[m_training:,:]
# Y_test = labels for the test set
Y_test =  Y[m_training:]

print("Number of samples in the train set:", X_training.shape[0])
print("Number of samples in the test set:", X_test.shape[0])
print("\nNumber of night instances in test:", np.sum(Y_test==-1))
print("Number of day instances in test:", np.sum(Y_test==1))

# standardize the input matrix
# the transformation is computed on training data and then used on all the 3 sets
scaler = preprocessing.StandardScaler().fit(X_training) 

np.set_printoptions(suppress=True) 
X_training = scaler.transform(X_training)
print ("Mean of the training input data:", X_training.mean(axis=0))
print ("Std of the training input data:",X_training.std(axis=0))

X_test =  scaler.transform(X_test)
print ("Mean of the test input data:", X_test.mean(axis=0))
print ("Std of the test input data:", X_test.std(axis=0))

2177
1632
Number of samples in the train set: 1632
Number of samples in the test set: 545

Number of night instances in test: 247
Number of day instances in test: 298
Mean of the training input data: [-0.  0. -0.]
Std of the training input data: [1. 1. 1.]
Mean of the test input data: [-0.0423764  -0.03894035 -0.00622458]
Std of the test input data: [1.02363564 1.04452829 0.99343475]


We **add a 1 in front of each sample** so that we can use a vector in **homogeneous coordinates** to describe all the coefficients of the model. This can be done with the function $hstack$ in $numpy$.

In [7]:
def to_homogeneous(X_training, X_test):
    # Add a 1 to each sample (homogeneous coordinates)
    X_training = np.hstack( [np.ones( (X_training.shape[0], 1) ), X_training] )
    X_test = np.hstack( [np.ones( (X_test.shape[0], 1) ), X_test] )
    
    return X_training, X_test

In [8]:
# convert to homogeneous coordinates using the function above
X_training, X_test = to_homogeneous(X_training, X_test)
print("Training set in homogeneous coordinates:")
print(X_training[:10])

Training set in homogeneous coordinates:
[[ 1.          0.13167195  0.11416158 -1.76557287]
 [ 1.         -0.01100106 -0.02993835 -1.83205964]
 [ 1.         -0.47850536 -0.47065089 -0.43929145]
 [ 1.          1.37749775  1.26543147  0.18204872]
 [ 1.          0.76351104  0.75258211  1.59774147]
 [ 1.         -0.48276064 -0.47246483 -0.11209462]
 [ 1.         -0.4807181  -0.47043845 -0.29446558]
 [ 1.         -0.49546975 -0.47383753  0.30621048]
 [ 1.         -0.34137177 -0.31414593 -1.27367176]
 [ 1.          0.30268115  0.27268638  0.17653713]]


**TO DO (A.1.2):** Now **complete** the function *perceptron*. <br>
The **perceptron** algorithm **does not terminate** if the **data** is not **linearly separable**, therefore your implementation should **terminate** if it **reached the termination** condition seen in class **or** if a **maximum number of iterations** have already been run, where one **iteration** corresponds to **one update of the perceptron weights**. In case the **termination** is reached **because** the **maximum** number of **iterations** have been completed, the implementation should **return the best model** seen throughout .

The input parameters to pass are:
- $X$: the matrix of input features, one row for each sample
- $Y$: the vector of labels for the input features matrix X
- $max\_num\_iterations$: the maximum number of iterations for running the perceptron

The output values are:
- $best\_w$: the vector with the coefficients of the best model (or the latest, if the termination condition is reached)
- $best\_error$: the *fraction* of misclassified samples for the best model

In [9]:
a = np.array([[+2,-3,-4,-5]])
pred = np.sign(a)
wrong = np.array([[True, False, True, False]])
ind = np.where(pred == -1)[1][0]
print(a)
print(pred)
print(wrong)
print(wrong.sum())
print(ind)

[[ 2 -3 -4 -5]]
[[ 1 -1 -1 -1]]
[[ True False  True False]]
2
1


In [10]:
def count_errors(current_w, X, Y):
    # This function:
    # computes the number of misclassified samples
    # returns the index of the first misclassified sample
  
    pred = np.sign(Y*np.matmul(current_w,X.transpose()))   
    wrong = pred<=0               
    num_misclassified = wrong.sum()             

    if num_misclassified > 0:        
        index_misclassified = np.where(wrong)[1][0]
    else:
        index_misclassified = -1 
    
    return num_misclassified, index_misclassified
        
def perceptron_update(current_w, x, y):
    # Place in this function the update rule of the perceptron algorithm
    # Remember that numpy arrays can be treated as generalized variables
    # therefore given array a = [1,2,3,4], the operation b = 10*a will yield
    # b = [10, 20, 30, 40]
    new_w = current_w + y*x
    return new_w

def perceptron(X, Y, max_num_iterations):
    
    # Initialize some support variables
    num_samples = X.shape[0]
    # best_errors will keep track of the best (minimum) number of errors
    # seen throughout training, used for the update of the best_w variable
    best_error = num_samples+1
    
    # Initialize the weights of the algorith with w=0
    curr_w = np.random.rand(1,X.shape[1])

    # The best_w variable will be used to keep track of the best solution
    best_w = curr_w.copy()

    # compute the number of misclassified samples and the index of the first of them
    num_misclassified, index_misclassified = count_errors(curr_w, X, Y)
    # update the 'best' variables
    if num_misclassified < best_error:
        best_error = num_misclassified
        best_w = curr_w
    
    # initialize the number of iterations
    num_iter = 0
    # Main loop continue until all samples correctly classified or max # iterations reached
    # Remember that to signify that no errors were found we set index_misclassified = -1
    while index_misclassified != -1 and num_iter < max_num_iterations:
        
        # Call the perceptron_update function using the found misclassified sample
        curr_w = perceptron_update(X[[index_misclassified],:],Y[[index_misclassified]],curr_w)
        
        # permute the dataset 
        # **IMPORTANT** use the SAME permutation for samples and labels
        per = np.random.permutation(num_samples)
        X = X[per]
        Y = Y[per]
        
        # repeat the error count and best variables update
        num_misclassified, index_misclassified = count_errors(curr_w, X, Y)
        # update the 'best' variables
        if num_misclassified < best_error:
            best_error = num_misclassified
            best_w = curr_w
        
        # update the iteration number
        num_iter += 1
    # as required, return the best error as a ratio with respect to the total number of samples
    best_error = best_error/num_samples
    
    return best_w, best_error


Now we use the implementation above of the perceptron to learn a model from the training data using 100 iterations and print the error of the best model we have found.

In [11]:
# Now run the perceptron for 100 iterations
w_found, error = perceptron(X_training,Y_training, 100)
print("Training Error of perceptron (100 iterations): " + str(error))

Training Error of perceptron (100 iterations): 0.044730392156862746


**TO DO (A.1.3):** use the best model $w\_found$ to **predict the labels for the test dataset** and print the fraction of misclassified samples in the test set (the test error that is an estimate of the true loss).

In [12]:
num_errors, _ = count_errors(w_found, X_test,Y_test)

true_loss_estimate = num_errors/X_test.shape[0]    
# NOTE: you can avoid using num_errors if you prefer, as long as true_loss_estimate is correct
print("Test Error of perceptron (100 iterations): " + str(true_loss_estimate))

Test Error of perceptron (100 iterations): 0.062385321100917435


**TO DO (A.Q1) [Answer the following]** <br>
What about the difference between the training error and the test error  in terms of fraction of misclassified samples? Explain what you observe. (Notice that with a very small dataset like this one results can change due to randomization, try to run with different random seeds if you get unexpected results).

italicized text<div class="alert alert-block alert-info">
**ANSWER A.Q1**:<br>
As a result of these 100 iterations, w_found is found to be the best weight. Based on the 100 iterations, the minimum error is **0.044730**, whereas the test error is **0.0623853** when using w_found. So the test error is smaller than training error after 100 iterations but they are still very close.
 </div>

**TO DO (A.1.4):** Copy the code from the last 2 cells above in the cell below and repeat the training with 3000 iterations. Then print the error in the training set and the estimate of the true loss obtained from the test set.

In [13]:
w_found, error = perceptron(X_training,Y_training, 1000) 
print("Training Error of perceptron (3000 iterations): " + str(error))

num_errors, _ =  count_errors(w_found, X_test,Y_test) 

true_loss_estimate = num_errors/X_test.shape[0] 
print("Test Error of perceptron (3000 iterations): " + str(true_loss_estimate))

Training Error of perceptron (3000 iterations): 0.03860294117647059
Test Error of perceptron (3000 iterations): 0.04220183486238532


**TO DO (A.Q2) [Answer the following]** <br>
What about the difference between the training error and the test error in terms of the fraction of misclassified samples) when running for a larger number of iterations? Explain what you observe and compare with the previous case.

<div class="alert alert-block alert-info">
**ANSWER A.Q2**:<br>

After **3000** iterations, training error it has a little change while test error is decreased.

</div>

# A.2) Logistic Regression
Now we use **logistic regression**, exploiting the implementation in **Scikit-learn**, to predict labels. We will also plot the decision boundaries of logistic regression.

We first load the dataset again.

In [14]:
# Load the dataset
X, Y = load_dset('lux.csv')

Header: ['rh', 'temp', 'uva', 'uvb', 'x', 'y', 'blueg', 'blueb', 'worn', 'pressure', 'timestamp', 'hour', 'minute', 'second', 'isnight']

Data shape: (2177, 15)

Dataset Example:
['23.846666666666668' '37.21333333333334' '5.577416666666666' '1.6235333333333333' '615.7777777777778' '686.0' '134.0' '117.0' '0.0' '1.000291690654385' '1650458376' '14' '39' '36' '0']

Number of samples loaded: 2177


**TO DO (A.2.1):** As for the previous part, **divide the data** into training and test (75%-25%) and **add a 1 as first component** to each sample.

In [15]:
# compute the splits
m_training = np.floor(0.75*X.shape[0]).astype(int)

# m_test is the number of samples in the test set (total-training)
m_test =  X.shape[0] - m_training

# X_training = instances for training set
X_training =  X[:m_training,:]
# Y_training = labels for the training set
Y_training =  Y[:m_training]

# X_test = instances for test set
X_test =   X[m_training:,:]
# Y_test = labels for the test set
Y_test =  Y[m_training:]
print("Number of samples in the train set:", X_training.shape[0])
print("Number of samples in the test set:", X_test.shape[0])
print("\nNumber of night instances in test:", np.sum(Y_test==-1))
print("Number of day instances in test:", np.sum(Y_test==1))

# standardize the input matrix
# the transformation is computed on training data and then used on all the 3 sets
scaler = preprocessing.StandardScaler().fit(X_training) 

np.set_printoptions(suppress=True)
X_training =  scaler.transform(X_training)
print ("Mean of the training input data:", X_training.mean(axis=0))
print ("Std of the training input data:",X_training.std(axis=0))

X_test =  scaler.transform(X_test)
print ("Mean of the test input data:", X_test.mean(axis=0))
print ("Std of the test input data:", X_test.std(axis=0))

# (***FIXED***)
X_training , X_test = to_homogeneous(X_training, X_test)

print(X_training[:10])

Number of samples in the train set: 1632
Number of samples in the test set: 545

Number of night instances in test: 265
Number of day instances in test: 280
Mean of the training input data: [-0.  0.  0.]
Std of the training input data: [1. 1. 1.]
Mean of the test input data: [-0.03269285 -0.03257518  0.0526219 ]
Std of the test input data: [0.92534277 0.92132474 1.01792479]
[[ 1.         -0.47130958 -0.452781    1.50101727]
 [ 1.         -0.47466887 -0.45306946  1.57937969]
 [ 1.         -0.46879011 -0.45321369  1.36057634]
 [ 1.         -0.29806621 -0.27900502 -1.21866016]
 [ 1.          0.21776475  0.18716542  0.96205727]
 [ 1.         -0.4144294  -0.37612423  1.13090718]
 [ 1.         -0.43696256 -0.41252036 -0.5836526 ]
 [ 1.         -0.1953174  -0.17420546  0.04590535]
 [ 1.          0.0775144   0.05267138 -1.645812  ]
 [ 1.          0.5233456   0.4695275   0.1762825 ]]


To define a logistic regression model in Scikit-learn use the instruction

$linear\_model.LogisticRegression(C=1e5)$

($C$ is a parameter related to *regularization*, a technique that
we will see later in the course. Setting it to a high value is almost
as ignoring regularization, so the instruction above corresponds to the
logistic regression you have seen in class.)

To learn the model you need to use the $fit(...)$ instruction and to predict you need to use the $predict(...)$ function. <br>
See the Scikit-learn documentation for how to use it [here](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html).

**TO DO (A.2.2):** **Define** the **logistic regression** model, then **learn** the model using **the training set** and **predict** on the **test set**. Then **print** the **fraction of samples misclassified** in the training set and in the test set.

In [16]:
# part on logistic regression for 2 classes
logreg = linear_model.LogisticRegression(C=1e5)

# learn from training set: hint use fit(...)
logreg.fit(X_training,Y_training)

print("Intercept:" , logreg.intercept_)
print("Coefficients:" , logreg.coef_)

# predict on training set
predicted_training = logreg.predict(X_training)

# print the error rate = fraction of misclassified samples
error_count_training = (predicted_training != Y_training).sum()
error_rate_training = error_count_training/m_training
print("Error rate on training set: "+str(error_rate_training))

# predict on test set
predicted_test = logreg.predict(X_test)

#print the error rate = fraction of misclassified samples
error_count_test = (predicted_test != Y_test).sum()
error_rate_test = error_count_test/m_test
print("Error rate on test set: " + str(error_rate_test))

Intercept: [-5.66395006]
Coefficients: [[ -5.62274398  44.25375704 -79.5338425    0.77713985]]
Error rate on training set: 0.0392156862745098
Error rate on test set: 0.04036697247706422


**TO DO (A.2.3)** Now **pick two features** and restrict the dataset to include only two features, whose indices are specified in the $idx0$ and $idx1$ variables below. Then split into training and test.

In [17]:
import random
# we define some additional variables:
#    feature_names  == name to display on the plots
#    feature_scales == scale (linear/log) to use for that metric
# this will be referred to in the following cell - to plot the values
feature_names  = ["UVA", "UVB", "Atm. Pressure"]
feature_scales = ["log", "log", "linear"]

# remember that we selected 3 sensors (uva, uvb, pressure), therefore
# to make the plot we need to reduce the data to 2D, so we choose two of them
# to do so change the following indices (between 0 and 2, inclusive)

ch = random.sample([0,1,2],2)
idx0 = ch[0]
idx1 = ch[1]

X_reduced = X[:,[idx0, idx1]]

# re-initialize the dataset splits, with the reduced sets
X_training = X_reduced[:m_training,:]
Y_training = Y[:m_training]

X_test = X_reduced[m_training:,:]
Y_test = Y[m_training:]

Now learn a model using the training data and measure the performances.

In [18]:
# learning from training data
logreg = linear_model.LogisticRegression(C=1e5)

logreg.fit(X_training,Y_training)

# print the error rate = fraction of misclassified samples
error_count_training = (predicted_training != Y_training).sum()
error_rate_test = error_count_training/m_training
print("Error rate on test set: " + str(error_rate_test))

Error rate on test set: 0.0392156862745098


**TO DO (A.Q3) [Answer the following]** <br>
Which features did you select and why? <br>
Compare the perfomance of the classifiers trained with every combination of two features with that of the baseline (which used all 3 features).

<div class="alert alert-block alert-info">
**ANSWER A.Q3**:<br>

The features I selected are **UVA** , **UVB** 

</div>

If everything is ok, the code below uses the model in $logreg$ to plot the decision region for the two features chosen above, with colors denoting the predicted value. It also plots the points (with correct labels) in the training set. It makes a similar plot for the test set.

In [19]:
#firstly we import an additional module to suppress warnings
import warnings
warnings.filterwarnings("ignore")

# Plot the decision boundary. For that, we will assign a color to each
# point in the mesh [x_min, x_max]x[y_min, y_max].
x_min, x_max = X_reduced[:, 0].min() - .5, X_reduced[:, 0].max() + .5
y_min, y_max = X_reduced[:, 1].min() - .5, X_reduced[:, 1].max() + .5
xx, yy = np.meshgrid(np.linspace(x_min, x_max, 500), np.linspace(y_min, y_max, 500))

Z = logreg.predict(np.c_[xx.ravel(), yy.ravel()])

# Put the result into a color plot
Z = Z.reshape(xx.shape)

# shift all the values to positive, to avoid problems with log-log plots
shift = .001 - X_reduced.min(axis=0)

fig, axs = plt.subplots(1, 2, figsize=(8,3))
axs[0].pcolormesh(xx + shift[0], yy + shift[1], Z, cmap=plt.cm.Set3)
axs[1].pcolormesh(xx + shift[0], yy + shift[1], Z, cmap=plt.cm.Set3)

# Plot also the training points
axs[0].scatter(X_training[:, 0] + shift[0], X_training[:, 1] + shift[1], c=Y_training, edgecolors='k', cmap=plt.cm.Set3)

axs[0].set_xlabel(feature_names[idx0])
axs[0].set_ylabel(feature_names[idx1])
axs[0].set_xscale(feature_scales[idx0])
axs[0].set_yscale(feature_scales[idx1])

axs[0].set_xlim(xx.min(), xx.max())
axs[0].set_ylim(yy.min(), yy.max())
axs[0].set_xticks(())
axs[0].set_yticks(())
axs[0].set_title('Training set')

# Plot also the test points 
axs[1].scatter(X_test[:, 0] + shift[0], X_test[:, 1] + shift[1], c=Y_test, edgecolors='k', cmap=plt.cm.Set3, marker='s')
axs[1].set_xlabel(feature_names[idx0])
axs[1].set_ylabel(feature_names[idx1])
axs[1].set_xscale(feature_scales[idx0])
axs[1].set_yscale(feature_scales[idx1])

axs[1].set_xlim(xx.min(), xx.max())
axs[1].set_ylim(yy.min(), yy.max())
axs[1].set_xticks(())
axs[1].set_yticks(())
axs[1].set_title('Test set')

plt.show()

# finally turn the warnings back on
warnings.filterwarnings("default")

<IPython.core.display.Javascript object>

# B) Linear Regression on the Atmospheric Conditions

As before, these **samples** were **extracted from** a Luxottica **I-SEE** device. <br>
For the **second part** of the laboratory we will **focus on linear regression**. We will try to estimate the **atmospheric pressure** starting from **temperature** and **relative humidity**.  (***FIXED***) 

First of all, we load again the dataset. Notice that the indices of the features were changed!

In [20]:
# Load the dataset
X, Y = load_dset('lux.csv', [0,1], 9, mode='reg')

Header: ['rh', 'temp', 'uva', 'uvb', 'x', 'y', 'blueg', 'blueb', 'worn', 'pressure', 'timestamp', 'hour', 'minute', 'second', 'isnight']

Data shape: (2177, 15)

Dataset Example:
['23.846666666666668' '37.21333333333334' '5.577416666666666' '1.6235333333333333' '615.7777777777778' '686.0' '134.0' '117.0' '0.0' '1.000291690654385' '1650458376' '14' '39' '36' '0']

Number of samples loaded: 2177


**TO DO (B.1)**: split the data in training and test sets (70%-30%)

In [21]:
# compute the splits
m_training = np.floor(0.75*X.shape[0]).astype(int)

# m_test is the number of samples in the test set (total-training)
m_test =  X.shape[0] - m_training

# X_training = instances for training set
X_training =  X[:m_training,:]
# Y_training = labels for the training set
Y_training =  Y[:m_training]

# X_test = instances for test set
X_test =   X[m_training:,:]
# Y_test = labels for the test set
Y_test =  Y[m_training:]

# standardize the input matrix
# the transformation is computed on training data and then used on all the 3 sets
scaler = preprocessing.StandardScaler().fit(X_training) 

np.set_printoptions(suppress=True)
X_training =  scaler.transform(X_training)
print ("Mean of the training input data:", X_training.mean(axis=0))
print ("Std of the training input data:",X_training.std(axis=0))

X_test =  scaler.transform(X_test)
print ("Mean of the test input data:", X_test.mean(axis=0))
print ("Std of the test input data:", X_test.std(axis=0))

Mean of the training input data: [0. 0.]
Std of the training input data: [1. 1.]
Mean of the test input data: [0.01614895 0.0074822 ]
Std of the test input data: [1.02304725 0.99551109]


# Model Training 

The model is trained (= estimated) minimizing the empirical error
$$
L_S(h) := \frac{1}{m_t} \sum_{z_i \in S_{t}} \ell(h,z_i)
$$
When the loss function is the quadratic loss
$$
\ell(h,z) := (y - h(x))^2
$$
we define  the Residual Sum of Squares (RSS) as
$$
RSS(h):= \sum_{z_i \in S_{t}} \ell(h,z_i) = \sum_{z_i \in S_{t}} (y_i - h(x_i))^2
$$ so that the training error becomes
$$
L_S(h) = \frac{RSS(h)}{m_t}
$$

We recal that, for linear models we have $h(x) = <w,x>$ and the Empirical error $L_S(h)$ can be written
in terms of the vector of parameters $w$ in the form
$$
L_S(w) = \frac{1}{m_t} \|Y - X w\|^2
$$
where $Y$ and $X$ are the matrices whose $i-$th row are, respectively, the output data $y_i$ and the input vectors $x_i^\top$. 

**TO DO (B.2):** compute the linear regression coefficients using np.linalg.lstsq from scikitlearn

In [22]:
#add a 1 at the beginning of each sample for training, and testing (use homogeneous coordinates)
X_trainingH, X_testH =  to_homogeneous(X_training,X_test)

print("Training set in homogeneous coordinates:")
print(X_trainingH[:10])

# Compute the least-squares coefficients using linalg.lstsq
w_np, RSStr_np, rank_Xtr, sv_Xtr = np.linalg.lstsq(X_trainingH, Y_training, rcond=None) 

print("LS coefficients with numpy lstsq:", w_np)

# compute Residual sums of squares 
Y_trainingR = Y_training.reshape(-1,1)  #***FIXED***  
RSStr_train = np.sqrt(np.sum((Y_trainingR-np.matmul(w_np,X_trainingH.transpose()))**2)) 

print("RSS with numpy lstsq: ", RSStr_np)
print("Empirical risk with numpy lstsq:", RSStr_train/m_training)

Training set in homogeneous coordinates:
[[ 1.          0.75554801 -0.01743097]
 [ 1.          0.07732787  0.14253758]
 [ 1.          1.2553175  -0.50227608]
 [ 1.         -1.2161071   0.22036487]
 [ 1.          0.76671395 -0.61522813]
 [ 1.         -1.10502101 -0.71561926]
 [ 1.         -0.65950505 -0.48897229]
 [ 1.          0.84168022 -0.34361442]
 [ 1.          0.99218628 -0.50932356]
 [ 1.          1.32744528 -0.4239969 ]]
LS coefficients with numpy lstsq: [ 0.99312408 -0.00317588 -0.00004887]
RSS with numpy lstsq:  [0.0092221]
Empirical risk with numpy lstsq: 0.005061463467685504


## Data prediction 

Compute the output predictions on both training and validation set.and compute the Residual Sum of Sqaures (RSS). 

**TO DO (B.3)**: Compute these quantities on  training, validation and test sets.

In [23]:
#compute predictions on training and validation

#prediction_training 
prediction_training = np.matmul(w_np,X_trainingH.transpose())
prediction_test =  np.matmul(w_np,X_testH.transpose())

prediction_training = np.dot(X_trainingH, w_np)
prediction_test = np.dot(X_testH, w_np)

#what about the loss for points in the test data?
RSS_test = np.sum((Y_test-predicted_test)**2)


print("RSS on test data:",  RSS_test)
print("Loss estimated from test data:", RSS_test/m_test)


RSS on test data: 973.419659664447
Loss estimated from test data: 1.7860911186503614


**TO DO (B.Q1) [Answer the following]** <br>
Comment on the results you get and on the difference between the train and test errors.

**bold text**<div class="alert alert-block alert-info">
**ANSWER B.Q1**:<br>

Loss estimated from test data: **1.7860911186503614**

</div>

In [24]:
#compute predictions on training and validation

#prediction_training 
prediction_training = np.matmul(w_np,X_trainingH.transpose())
prediction_test =  np.matmul(w_np,X_testH.transpose())

#what about the loss for points in the test data?
RSS_test = np.sum((Y_test-predicted_test)**2)

print("RSS on test data:",  RSS_test)
print("Loss estimated from test data:", RSS_test/m_test)

RSS on test data: 973.419659664447
Loss estimated from test data: 1.7860911186503614


In [25]:
#compute predictions on training and validation

#prediction_training 
prediction_training = np.matmul(w_np,X_trainingH.transpose()) 
prediction_test =  np.matmul(w_np,X_testH.transpose())

#what about the loss for points in the test data?
RSS_test = np.sum((Y_test-predicted_test)**2)

print("RSS on test data:",  RSS_test)
print("Loss estimated from test data:", RSS_test/m_test)

RSS on test data: 973.419659664447
Loss estimated from test data: 1.7860911186503614


Now let's plot the data and check our estimation result.

In [26]:
fig = plt.figure(figsize=(10,18))

ax1 = fig.add_subplot(1, 2, 1, projection='3d')
ax1.scatter(X_training[:,0], X_training[:,1], Y_training, label="Input Data")
ax1.scatter(X_training[:,0], X_training[:,1], prediction_training, label="Prediction Data")
ax1.set_xlabel("Norm. Temperature")
ax1.set_ylabel("Norm. Relative Humidity")
ax1.set_zlabel("Norm. Atm. Pressure")
ax1.set_title("Training Set")
ax1.legend()

ax2 = fig.add_subplot(1, 2, 2, projection='3d')
ax2.scatter(X_test[:,0], X_test[:,1], Y_test, label="Input Data")
ax2.scatter(X_test[:,0], X_test[:,1], prediction_test, label="Prediction Data")
ax2.set_xlabel("Norm. Temperature")
ax2.set_ylabel("Norm. Relative Humidity")
ax2.set_zlabel("Norm. Atm. Pressure")
ax2.set_title("Test Set")
ax2.legend()
plt.show()

<IPython.core.display.Javascript object>

## Ordinary Least-Squares using scikit-learn
Another fast way to compute the LS estimate is through sklearn.linear_model

In [27]:
LinReg = linear_model.LinearRegression() 
LinReg.fit(X_training, Y_training)  
print("Intercept:", LinReg.intercept_)
print("Least-Squares Coefficients:", LinReg.coef_)
prediction_training = LinReg.predict(X_training)  
prediction_test = LinReg.predict(X_test)  
print("Measure on training data:", 1-LinReg.score(X_training, Y_training))

Intercept: 0.9931240750203619
Least-Squares Coefficients: [-0.00317588 -0.00004887]
Measure on training data: 0.36142891329678095
